In [1]:
import sys
import os
if 'google.colab' in sys.modules:
    # Google Drive
    from google.colab import drive
    drive.mount("/content/drive")
    %cd /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks
    
    
    # Kaggle API
    !pip install --upgrade --force-reinstall --no-deps kaggle
    import json
    f = open("/content/drive/MyDrive/colab_notebooks/kaggle/kaggle.json", "r")
    json_data = json.load(f)
    os.environ["KAGGLE_USERNAME"] = json_data["username"]
    os.environ["KAGGLE_KEY"] = json_data["key"]
    

    # Input File Location
    inname = "localnb003-spacy-dataset"
    if not os.path.exists(inname):
        raise ValueError(f"Folder '{inname}' does not exist!")
    
    # Output File Location
    dname = "localnb004-spacy-train"
    if not os.path.exists(dname):
        os.makedirs(dname)
    
    # GPU
    !nvidia-smi

Mounted at /content/drive
/content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks
     |████████████████████████████████| 61kB 3.2MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.12-cp37-none-any.whl size=73053 sha256=b9489be860f79e96703e144a63801f26c746bd289e28783e2eb24d23af174016
  Stored in directory: /root/.cache/pip/wheels/a1/6a/26/d30b7499ff85a4a4593377a87ecf55f7d08af42f0de9b60303
Successfully built kaggle
  Found existing installation: kaggle 1.5.12
    Uninstalling kaggle-1.5.12:
      Successfully uninstalled kaggle-1.5.12
Wed Jul 14 22:50:46 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.42.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M.

In [2]:
import re
import json
import time
import datetime
import random
import glob
import importlib

import numpy as np
import pandas as pd

from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import torch 
if torch.cuda.is_available():
    import cupy
import gc 

In [3]:
!ls ../input/spacy3

catalogue-2.0.3-py3-none-any.whl
en_core_web_lg-3.0.0-py3-none-any.whl
en_core_web_md-3.0.0-py3-none-any.whl
en_core_web_sm-3.0.0-py3-none-any.whl
en_core_web_trf-3.0.0-py3-none-any.whl
pathy-0.5.2-py3-none-any.whl
pydantic-1.7.3-cp37-cp37m-manylinux2014_x86_64.whl
smart_open-3.0.0-py3-none-any.whl
spacy-3.0.6-cp37-cp37m-manylinux2014_x86_64.whl
spacy_alignments-0.8.3-cp37-cp37m-manylinux2014_x86_64.whl
spacy_legacy-3.0.5-py2.py3-none-any.whl
spacy_transformers-1.0.2-py2.py3-none-any.whl
srsly-2.4.1-cp37-cp37m-manylinux2014_x86_64.whl
thinc-8.0.3-cp37-cp37m-manylinux2014_x86_64.whl
typer-0.3.2-py3-none-any.whl


In [4]:
%%time
!pip uninstall fastai en-core-web-sm en-core-web-lg spacy -y -q
!pip install ../input/spacy3/catalogue-2.0.3-py3-none-any.whl ../input/spacy3/typer-0.3.2-py3-none-any.whl ../input/spacy3/srsly-2.4.1-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/pathy-0.5.2-py3-none-any.whl ../input/spacy3/smart_open-3.0.0-py3-none-any.whl ../input/spacy3/pydantic-1.7.3-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/thinc-8.0.3-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/spacy-3.0.6-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/spacy_legacy-3.0.5-py2.py3-none-any.whl -q
!pip install ../input/spacy3/en_core_web_lg-3.0.0-py3-none-any.whl ../input/spacy3/en_core_web_md-3.0.0-py3-none-any.whl ../input/spacy3/en_core_web_sm-3.0.0-py3-none-any.whl -q
!pip install ../input/spacy3/spacy_alignments-0.8.3-cp37-cp37m-manylinux2014_x86_64.whl ../input/spacy3/spacy_transformers-1.0.2-py2.py3-none-any.whl ../input/spacy3/en_core_web_trf-3.0.0-py3-none-any.whl -q

     |████████████████████████████████| 2.1MB 4.3MB/s 
     |████████████████████████████████| 3.3MB 69.5MB/s 
     |████████████████████████████████| 901kB 70.9MB/s 
CPU times: user 346 ms, sys: 105 ms, total: 451 ms
Wall time: 1min 13s


In [5]:
!pip install thinc==8.0.7

     |████████████████████████████████| 624kB 4.1MB/s 
     |████████████████████████████████| 10.1MB 29.5MB/s 
ERROR: spacy 3.0.6 has requirement pydantic<1.8.0,>=1.7.1, but you'll have pydantic 1.8.2 which is incompatible.
  Found existing installation: pydantic 1.7.3
    Uninstalling pydantic-1.7.3:
      Successfully uninstalled pydantic-1.7.3
  Found existing installation: catalogue 2.0.3
    Uninstalling catalogue-2.0.3:
      Successfully uninstalled catalogue-2.0.3
  Found existing installation: thinc 8.0.3
    Uninstalling thinc-8.0.3:
      Successfully uninstalled thinc-8.0.3


In [6]:
import spacy
assert spacy.__version__ == '3.0.6'
import en_core_web_trf
import torch 
from spacy.tokens import DocBin
import srsly

In [7]:
#!python -m spacy init fill-config ../input/spacy-train-set/base_config2.cfg config.cfg
!python -m spacy init fill-config ./{inname}/base_config2.cfg config.cfg

2021-07-14 22:53:57.630058: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
 # CV 0 (1)
!python -m spacy train config.cfg --output ./{dname}/cv0  --paths.train ./{inname}/cv_train_0.spacy --paths.dev ./{inname}/cv_valid_0.spacy   --gpu-id 0 

2021-07-14 22:54:35.642872: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Created output directory: localnb004-spacy-train/cv0
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-07-14 22:54:38,274] [INFO] Set up nlp object from config
[2021-07-14 22:54:38,284] [INFO] Pipeline: ['transformer', 'ner']
[2021-07-14 22:54:38,288] [INFO] Created vocabulary
[2021-07-14 22:54:38,288] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 419kB/s]
Downloading: 100% 899k/899k [00:00<00:00, 1.68MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 1.30MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 1.94MB/s]
Downloading: 100% 501M/501M [00:07<00:00, 69.3MB/s]
[2021-07-14 22:55:15,658] [INFO] Initialized pipeline components: ['transformer', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeli

In [ ]:
# CV 1 (2)
!python -m spacy train config.cfg --output ./{dname}/cv1  --paths.train ./{inname}/cv_train_1.spacy --paths.dev ./{inname}/cv_valid_1.spacy   --gpu-id 0 

In [ ]:
# CV 2 (3)
#!python -m spacy train config.cfg --output ./output/cv0-DA  --paths.train ../input/spacy-train-set/cv_train_0_DA.spacy --paths.train ../input/spacy-train-set/cv_train_0.spacy  --paths.dev ../input/spacy-train-set/cv_valid_0.spacy   --gpu-id 0 
!python -m spacy train config.cfg --output ./{dname}/cv2  --paths.train ./{inname}/cv_train_2.spacy --paths.dev ./{inname}/cv_valid_2.spacy   --gpu-id 0 

2021-07-13 22:35:36.790777: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-07-13 22:35:39,303] [INFO] Set up nlp object from config
[2021-07-13 22:35:39,314] [INFO] Pipeline: ['transformer', 'ner']
[2021-07-13 22:35:39,319] [INFO] Created vocabulary
[2021-07-13 22:35:39,319] [INFO] Finished initializing nlp object
Downloading: 100% 481/481 [00:00<00:00, 455kB/s]
Downloading: 100% 899k/899k [00:00<00:00, 3.48MB/s]
Downloading: 100% 456k/456k [00:00<00:00, 2.69MB/s]
Downloading: 100% 1.36M/1.36M [00:00<00:00, 4.14MB/s]
Downloading: 100% 501M/501M [00:07<00:00, 64.6MB/s]
[2021-07-13 22:36:13,649] [INFO] Initialized pipeline components: ['transformer', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['transformer', 'ner']
ℹ Initial learn rate: 0.0
E 

In [ ]:
#!python -m spacy train config.cfg --output ./output/cv2_3s --paths.train ../input/spacy-train-set/cv_train_2_3s.spacy  --paths.dev ../input/spacy-train-set/cv_valid_2.spacy  --gpu-id 0 #--code ../input/spacy-train-set/function.pyn -m spacy train config.cfg --output ./output/cv4 --paths.train ../input/spacy-train-set/train_sent.spacy  --paths.dev ../input/spacy-train-set/train_sent.spacy  --gpu-id 0 #--code ../input/spacy-train-set/function.py

In [ ]:
#!python -m spacy train config.cfg --output ./output/cv3 --paths.train ../input/spacy-train-set/cv_train_3.spacy  --paths.dev ../input/spacy-train-set/cv_valid_3.spacy  --gpu-id 0 #--code ../input/spacy-train-set/function.pyn -m spacy train config.cfg --output ./output/cv4 --paths.train ../input/spacy-train-set/train_sent.spacy  --paths.dev ../input/spacy-train-set/train_sent.spacy  --gpu-id 0 #--code ../input/spacy-train-set/function.py

# Upload to Kaggle Dataset

In [ ]:
%cd /content/drive/MyDrive/colab_notebooks/kaggle/Kaggle-Coleridge-Initiative/notebooks

In [ ]:
dname = "localnb004-spacy-train"
# !kaggle datasets init -p {dname}

# with open(f"{dname}/dataset-metadata.json", "r") as jsonFile:
#     data = json.load(jsonFile)

# data["id"] = f"riow1983/{dname}"
# data["title"] = dname

# with open(f"{dname}/dataset-metadata.json", "w") as jsonFile:
#     json.dump(data, jsonFile)

# !kaggle datasets create -p {dname} --dir-mode zip

In [ ]:
# Update datasets
kaggle datasets version -p {dname} --dir-mode tar -m "Updated data"